## Homework assignment 1: Text Suggestion

__Soft deadline: 27.09 23:59__   
__Hard deadline: 30.09 23:59__

### About task

In this assignment, you'll implement a system that helps in text typing by suggesting continuations of a word or several subsequent words in real time, similar to those used in email applications or search engines. For extra credit, you'll need to wrap your system in a user interface using the [reflex](https://github.com/reflex-dev/reflex) library or similar tools. This assignment won't require training any models; we'll stick to n-gram generation.

### Structure

This homework assignment consists of two parts: the main part and a bonus part. In the first part, you'll need to complete five tasks, which will yield a minimally viable solution. In the second part, you'll use what you've already accomplished to implement a fully functional text suggestion system with a user interface. In the second part, we won't limit your imagination in any way. Do whatever you can, as long as you end up with a usable framework. The better your results, the more points you'll receive. If you do really well, we'll add bonuses at our discretion.

### Assessment and penalties

The maximum grade allowed for this assignment is 15 points. Submitting an assignment after the hard deadline is prohibited. Submitting a solution after the soft deadline will deduct __one__ point for each day late.

The assignment must be completed independently. "Similar" solutions are considered plagiarism, and all students involved (including those from whom the plagiarism was committed) cannot receive more than 0 points for it. All code must be written independently. Using someone else's code is prohibited, even with a link to the source. Within reasonable limits, of course. Taking a couple of obvious lines of code to implement a small feature is acceptable.

Ineffective code implementation may negatively impact your grade. Your grade may also be reduced for poorly readable code.

When submitting an assignment, you will be required to submit all the code, and if you choose the bonus section, you will also be required to submit a report and a video demonstrating your UI. For the main part you can get up to __10__ points, and for the bonus part – up to __5__ points.

### Data

To obtain text statistics, use the emails.csv dataset. You can find it at [link](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). It contains over 500,000 emails in English.

In [1]:
import pandas as pd

emails = pd.read_csv('emails.csv')
len(emails)

Note that the data is very dirty. Each email contains various meta-information, which will only interfere with predicting the rest of the text.

__Task 1 (2 points).__ Clean the text corpus as you see fit and explain your choice. Ideally, the processed texts should contain only the text of the email itself and nothing extraneous, such as links, recipients, or other characters that we definitely don't want to continue the text.

In [5]:
# your code here

For the next task, you'll need to tokenize the text. To do this, simply break it up into words. Obviously, the end result for the user will be better if your system also suggests appropriate punctuation. However, if you notice that this is degrading the quality of the text itself, you can remove all non-alphabetic characters during the tokenization step.

## General solution framework

We want to create a system that will speed up typing by suggesting suitable continuations. We'll use an n-gram model to suggest the next word. Since the n-gram model works with whole words, and we want to provide suggestions in real time even when the word isn't yet complete, we first need to learn how to complete a word.

## Word Completion

In this section, you will implement a method for completing a word to a whole word based on its beginning (prefix). To do this, you first need to find all words with a certain prefix. We will call the search function for matching words after each letter typed by the user. Therefore, it is very important for the search to be as fast as possible. A simple search through all words takes $O(|V| \cdot n)$ time, where $|V|$ is the vocabulary size and $n$ is the prefix length. We will write a [prefix tree](https://en.wikipedia.org/wiki/Trie), which allows searching for words in no more than $O(n + mk)$, where $m$ is the number of matching words and $k$ is the suffix length.

__Task 2 (2 points).__ Complete the prefix tree to search for words by prefix.

In [401]:
from typing import List

class PrefixTreeNode:
    def __init__(self):
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: list of all unique tokens in the corpus
        """
        self.root = PrefixTreeNode()
        
        # your code here

    def search_prefix(self, prefix) -> List[str]:
        """
        Returns all words that begin with prefix
        prefix: str – word's prefix
        """

        # your code here

In [ ]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Now that we have a way to quickly find all words with a certain prefix, we need to sort them by probability to select the best ones. We'll estimate a word's probability based on its occurrence in the corpus.

__Task 3 (2 points)__. Complete the `WordCompletor` class, which generates a vocabulary and prefix tree and can also find all possible continuations of a word along with their probabilities. In this class, you can further filter words if needed, for example, by removing all the rarest ones. Try to optimize your code as much as possible.

In [ ]:
class WordCompletor:
    def __init__(self, corpus):
        """
        corpus: list – corpus of texts
        """
        # your code here
        self.prefix_tree = PrefixTree(<vocabulary>)

    def get_words_and_probs(self, prefix: str) -> (List[str], List[float]):
        """
        Returns a list of words starting with prefix
        with their probabilities (nothing needs to be normalized)
        """
        words, probs = [], []
        # your code here
        return words, probs

In [ ]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Next word predictiong

Now that we can complete a user's word, we can go further and suggest the next word (or several) based on the completed word. For this, we'll use an n-gram model.

Recall that the sequence probability for such a model is calculated using the formula
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

$P(w_i \mid w_{i-1}, \dots, w_{i-n})$ is estimated by the occurrence of the n-gram. 

__Task 4 (2 points).__ Write a class for the n-gram model. No smoothing is needed, as we don't want the model to recommend random words (even if it does very rarely).

In [403]:
class NGramLanguageModel:
    def __init__(self, corpus, n):
        # your code here

    def get_next_words_and_probs(self, prefix: list) -> (List[str], List[float]):
        """
        Returns a list of words that can follow the prefix,
        as well as a list of probabilities for these words.
        """

        next_words, probs = [], []
        
        # your code here

        return next_words, probs

In [ ]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Great, we can now combine the two methods into an automatic text completion tool: the first will complete a word, and the second will suggest continuations. We'd like a list of possible continuations to be offered, from which the user can choose the most suitable one. The hardest part here is carefully choosing what to show and what not.

__Task 5 (2 points).__ As a baseline, implement a method that always returns the most probable continuation using a greedy method. After that, you can add the option to generate multiple continuation options, which will make the method much better.

In [443]:
class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, n_texts=1) -> List[List[str]]:
        """
        Returns possible text continuation options (only one by default)
        
        text: a string or word list – the text written by the user
        n_words: the number of words to be completed by the n-gram model
        n_texts: the number of continuations returned (currently only one)
        
        return: List[List[srt]] – a list of n_texts word lists, each containing 1 + n_words words
        The first word is the one completed by WordCompletor.
        """

        suggestions = []

        # your code here

        return suggestions

In [ ]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [450]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

## Bonus Part: Adding UI

Running cells in Jupiter is great, but it would be better if your solution was actually usable. To achieve this, we suggest adding a full-fledged User Interface. We recommend using [reflex](https://github.com/reflex-dev/reflex) for this. It's a Python library for creating web interfaces with very rich functionality.

Your task is to create a text input field in which suggestions will appear in real time as the user types. Consider how the user will select suggestions, how many continuations to recommend, and so on. Overall, it should look nice and user-friendly. In this section, you can modify all the classes as you see fit and add any heuristics. If necessary, you can additionally process the text and generally do whatever you deem necessary.

You can earn up to __5 bonus points__ for this assignment, depending on how well and conveniently you implemented your system. When submitting the assignment, please attach a short __report__ (half a page) describing your system, as well as a __video__ (1-2 minutes) demonstrating the interface.

We strongly recommend that you format the code as a project rather than writing it in a notebook. However, if you really want to write it here, at least don't change the code in previous assignments so that it can be properly graded.